Implementation of Arima using the ibex_ARIMA class in `src.arima`

In [1]:
# Standard Imports
import importlib
import os
import numpy as np
import pandas as pd
import sys

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Src modules
sys.path.append(os.path.dirname(os.getcwd())) # Add the parent directory to the Python path so we can import src modules
import src
from src.arima import arima_trial, ibex_ARIMA
from src.data_setup import get_data, train_val_split, create_day_of_week, RAW_PATH, RESULTS_PATH
from src.model_evaluation import model_eval_pipeline, eval_hypothesis_test, transform_daily_sales_predictions
from src.visualisation import plot_heatmap, plot_sales_by, plot_time_series_preds, plot_rolling_average_stdev

In [2]:
train, test, stores, transactions = get_data()
# merge stores data into train
train = train.merge(stores, on ='store_nbr')
assert train.isnull().any().any() == False
assert train.duplicated().any() == False
print('no null or duplicate values in the training data')

loading pickled dataframes...
no null or duplicate values in the training data


In [3]:
# split into train and validation data
train, validation = train_val_split(train)
# validation = validation.groupby('date')['sales'].sum().reset_index()
train.shape, validation.shape

((2949210, 25), (51678, 25))

In [4]:
importlib.reload(src.arima)
from src.arima import ibex_ARIMA

In [5]:
arima = ibex_ARIMA(train)
# basic_arima.plot_autocorrelation()
# basic_arima.test_stationarity()

In [6]:
arima.fit((1, 1, 1), (1, 1, 1, 7), plot_summary=False)

In [7]:
val = arima.evaluate(validation=validation.groupby('date')['sales'].sum().reset_index())

In [8]:
val

,date,sales,day_of_week,pred_diff_sales,pred_sales
0,2017-07-18,7.301337e+05,2,-79304.309105,7.390212e+05
1,2017-07-19,7.679788e+05,3,31470.695726,7.704919e+05
2,2017-07-20,6.882881e+05,4,-133149.956091,6.373419e+05
3,2017-07-21,7.824183e+05,5,146184.255046,7.835262e+05
4,2017-07-22,9.329021e+05,6,236679.372029,1.020206e+06
5,2017-07-23,1.024289e+06,7,96954.549813,1.117160e+06
6,2017-07-24,8.165643e+05,1,-311570.117048,8.055900e+05
7,2017-07-25,7.135816e+05,2,-70624.491944,7.349655e+05
8,2017-07-26,7.406531e+05,3,28274.542886,7.632400e+05
9,2017-07-27,6.598498e+05,4,-131729.048438,6.315110e+05


In [23]:
val = val.drop(columns=['pred_diff_sales'])
val


,date,sales,day_of_week,pred_sales
0,2017-07-18,7.301337e+05,2,7.390212e+05
1,2017-07-19,7.679788e+05,3,7.704919e+05
2,2017-07-20,6.882881e+05,4,6.373419e+05
3,2017-07-21,7.824183e+05,5,7.835262e+05
4,2017-07-22,9.329021e+05,6,1.020206e+06
5,2017-07-23,1.024289e+06,7,1.117160e+06
6,2017-07-24,8.165643e+05,1,8.055900e+05
7,2017-07-25,7.135816e+05,2,7.349655e+05
8,2017-07-26,7.406531e+05,3,7.632400e+05
9,2017-07-27,6.598498e+05,4,6.315110e+05


In [25]:
train.head()

,date,store_nbr,family,sales,onpromotion,year,month,week,day_of_week,day_of_month,...,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season,city,state,type,cluster
0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,2013,1,1,2,1,...,0,1,0,1,0,0,Quito,Pichincha,D,13
1,2013-01-01,1,BABY CARE,0.0,0.0,2013,1,1,2,1,...,0,1,0,1,0,0,Quito,Pichincha,D,13
2,2013-01-01,1,BEAUTY,0.0,0.0,2013,1,1,2,1,...,0,1,0,1,0,0,Quito,Pichincha,D,13
3,2013-01-01,1,BEVERAGES,0.0,0.0,2013,1,1,2,1,...,0,1,0,1,0,0,Quito,Pichincha,D,13
4,2013-01-01,1,BOOKS,0.0,0.0,2013,1,1,2,1,...,0,1,0,1,0,0,Quito,Pichincha,D,13


In [26]:
importlib.reload(src.model_evaluation)
from src.model_evaluation import transform_daily_sales_predictions

val = transform_daily_sales_predictions(val, train)
val.head()

,date,sales,day_of_week,pred_sales,store_nbr,family,pct_sales,transformed_sales
0,2017-07-18,730133.6875,2,739021.190895,1,AUTOMOTIVE,6.798028e-06,5.023887
1,2017-07-18,730133.6875,2,739021.190895,1,BABY CARE,0.000000e+00,0.000000
2,2017-07-18,730133.6875,2,739021.190895,1,BEAUTY,4.529532e-06,3.347420
3,2017-07-18,730133.6875,2,739021.190895,1,BEVERAGES,2.892669e-03,2137.743920
4,2017-07-18,730133.6875,2,739021.190895,1,BOOKS,2.611756e-07,0.193014


In [21]:
model_eval_pipeline(validation['sales'], a['transformed_pred_sales'])

{'mae': 336.5845219207806,
 'mse': 1118342.995234885,
 'rmse': 1057.5173734908024,
 'rmsle': 1.1717345176616767,
 'r2': 0.2998583776179432}